<a href="https://colab.research.google.com/github/ish66726-a11y/Quant-Finance-Fundamentals/blob/main/notebooks/02%E5%82%B5%E5%88%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2 債券

#2-1 債券の基礎

## 債券とは
- 投資家が企業や政府にお金を貸すことで成立する「負債投資」。
- 発行体は一定期間資金を借り、利子を支払う。

---

## 債券の種類

#### 1. ゼロクーポン債（Zero Coupon Bond）
- 元本のみ返済、利子なし。
- 価格（現在価値）:
  $$P = \frac{X}{(1+r)^N}$$
  - P：債券価格（現在価値）
  - X：額面（満期時に受け取る元本）
  - r：市場金利（1期あたりの割引率）
  - N：満期までの期数（年1回複利なら年数）
- 例：額面 1000円、満期 2年、市場金利 4%
  → 価格  ≒ 924円

#### 2. クーポン債（Coupon Bond）
- 毎年クーポン（利子）を支払い、満期時に元本も返済。
- 価格（現在価値）:
  $$P = \sum_{t=1}^N \frac{C}{(1+r)^t} + \frac{X}{(1+r)^N}$$
  または  
  $$P = \frac{C}{r}\left(1 - \frac{1}{(1+r)^N}\right) + \frac{X}{(1+r)^N}$$
  - P：債券価格（現在価値）
  - C：各期に支払われるクーポン額   $(C = \text{額面} \times \text{クーポン率})$
  - X：額面（満期時に返済される元本）
  - r：市場金利（1期あたりの割引率）
  - N：満期までの期数
  - t：クーポン支払時点を表すインデックス
- 例：額面 1000円、クーポン率 10%、満期 3年、市場金利4%
  → 価格 ≒ 1167円

※ 連続複利を用いる場合は、割引因子は $(1+r)^t$ の代わりに $e^{rt}$ となる。

※ ここでは金利 $r$ が満期まで一定であると仮定している。
  実際の市場では金利は時間とともに変動するため、
  10章でVasicekモデルを導入する。

---

👉 債券価格は「将来キャッシュフローの現在価値の合計」で決まる。  
👉 計算時は **市場金利** を使う点が重要。

---

## Python実装例:債券価格

In [2]:
# ゼロクーポン債の価格を計算する関数
def calculate_zero_coupon_price(principal, maturity, interest_rate):
    r = interest_rate / 100  # %を小数に変換
    return principal / ((1 + r) ** maturity)


# ===== 実行部分 =====

price = calculate_zero_coupon_price(1000, 2, 4)
print(f"債券価格 = {price:.2f}円")


債券価格 = 924.56円


In [ ]:
# クーポン債の価格を計算する関数
def calculate_coupon_bond_price(principal, coupon_rate, maturity, interest_rate):
    r = interest_rate / 100      # 市場金利
    c = coupon_rate / 100        # クーポン利率
    coupon = principal * c

    price = 0

    # 各年のクーポンを割引
    for t in range(1, maturity + 1):
        price += coupon / ((1 + r) ** t)

    # 満期時の元本を割引
    price += principal / ((1 + r) ** maturity)

    return price


# ===== 実行部分 =====

price = calculate_coupon_bond_price(1000, 10, 3, 4)
print(f"債券価格 = {price:.2f}円")


債券価格 = 1166.51円


#2-2 債券の利回りと最終利回り（YTM）



##2-1との違い：計算の「向き」
  - 2-1（価格算出）: 外部から与えられた市場金利 $r$ を用いて、債券の「適正価格 $P$」を求める。（順問題：$r \rightarrow P$）
  - 2-2（利回り算出）: 実際に市場でついている価格 $P$ から、その債券に投資した場合の「真の収益率 $y$」を逆算する。（逆問題：$P \rightarrow y$）

本節では、この逆算された利回り $y$（最終利回り）の定義と、その数値的な算出方法について学びます。

---
## 利回り（Yield）
- 投資収益を示す指標。
- 計算式：
  $$\text{Yield} = \frac{\text{年間クーポン支払額}}{\text{債券価格}}$$
- 例：価格 1,167円、年間クーポン 100円 → Yield ≒ 8.6%


---
## 最終利回り（Yield to Maturity, YTM）

- 債券を満期まで保有した場合の「実質利回り」。
- 内部収益率(IRR) に相当。


###**定義（離散時間・年1回複利）**

$$
P = \sum_{t=1}^{N} \frac{C}{(1+y)^t} + \frac{X}{(1+y)^N}
$$

- $P$：債券価格（現在の市場価格、既知）
- $C$：各期に支払われるクーポン額  （$C = \text{額面} \times \text{クーポン率}$）
- $X$：額面（満期時に返済される元本）
- $y$：最終利回り（YTM、1期あたりの利回り、未知関数）
- $N$：満期までの期数（年1回払いなら年数）
- $t$：クーポン支払時点を表すインデックス

YTM は「債券価格と将来キャッシュフローの現在価値を一致させる割引率」であり、一般に解析的に解けないため、数値計算（ニュートン法など）で求められる。

YTM は次の非線形方程式の解である：

$$
f(y) =
P - \sum_{t=1}^N \frac{C}{(1+y)^t} - \frac{X}{(1+y)^N} = 0
$$

この方程式を満たす y を求める問題になる。

※実務では通常、離散複利ベースでYTMを算出する。連続複利表示は、理論モデルや金利モデル分析で用いられる。

※ 連続複利を用いる場合、割引因子 $(1+y)^t$ は
  $e^{y t}$ に置き換わる。

###**Yield と YTM の違い**

| 項目 | Yield（単純利回り） | YTM（最終利回り） |
|------|--------------------|------------------|
| 計算対象 | 年間クーポンのみ | すべてのCF |
| 元本考慮 | × | 〇 |
| 再投資仮定 | なし | あり |
| 数学的性質 | 単純な比率 | IRR（内部収益率） |
| 精度 | 近似的 | 理論的に正確 |

👉 実務や理論分析で重要なのは YTM。

👉 Yield = 単純利回り、YTM = 実際の内部収益率。

---

## 価格とYTMの関係

債券価格とYTMは逆方向に動く。

- 価格 ↑ → YTM ↓
- 価格 ↓ → YTM ↑

理由：
YTMは「価格を現在価値と一致させる割引率」であるため、
価格が高いほど必要な割引率は低くなる。

---

## イールドカーブ（Yield Curve）
- 縦軸：利回り、横軸：満期（年数）。
- 通常は右上がりの形状をとる。
- 理由：
  - 将来の短期金利上昇に対する市場の期待
  - 長期債に固有の期間プレミアム（価格変動・流動性リスク）
  - インフレ不確実性

  
👉 イールドカーブは「期間が長いほど利回りが高い」傾向を示す。

👉 通常は右上がりだが、景気後退局面では逆イールド（右下がり）になることもある。

---

## Python実装例:ニュートン法によるYTM

In [7]:
# ==============================
# 債券の設定
# ==============================

P = 95.0   # 市場価格
X = 100.0  # 額面
C = 5.0    # クーポン（5%）
N = 5      # 満期（5年）

# ==============================
# 方程式 f(y)
# ==============================

def f(y):
    pv_coupons = sum(C / (1 + y)**t for t in range(1, N+1))
    pv_face = X / (1 + y)**N
    return P - pv_coupons - pv_face


# ==============================
# 導関数 f'(y)
# ==============================

def df(y):
    d_pv_coupons = sum(
        -t * C / (1 + y)**(t+1)
        for t in range(1, N+1)
    )
    d_pv_face = -N * X / (1 + y)**(N+1)
    return - d_pv_coupons - d_pv_face


# ==============================
# ニュートン法
# ==============================

def newton_method(initial_guess=0.05, tol=1e-10, max_iter=20):
    y = initial_guess

    print("初期値 =", y)

    for i in range(1, max_iter+1):
        y_new = y - f(y)/df(y)
        print(f"{i}回目: y = {y_new}")

        if abs(y_new - y) < tol:
            print(f"{i}回の反復で収束しました。")
            return y_new

        y = y_new

    print("収束しませんでした。")
    return None


# ==============================
# 実行
# ==============================

print("=== YTMをニュートン法で求めます ===")
ytm = newton_method(initial_guess=0.05)

if ytm is not None:
    print("\n計算結果")
    print("求められたYTM =", ytm)
    print("年利換算 =", ytm * 100, "%")

=== YTMをニュートン法で求めます ===
初期値 = 0.05
1回目: y = 0.06154873990641338
2回目: y = 0.06193188114819676
3回目: y = 0.06193228268107686
4回目: y = 0.0619322826815172
4回の反復で収束しました。

計算結果
求められたYTM = 0.0619322826815172
年利換算 = 6.19322826815172 %


#2-3 市場金利と債券価格の関係
## 基本原則
- 債券価格と市場金利は **逆相関**。

| 金利 | 債券価格 |
|---------------|----------------|
| 上昇 | 下落|
| 下落 | 上昇|
---
## 債券価格と金利

- ゼロクーポン債
  - 価格：
  $$P = \frac{X}{(1+r)^N}$$
- クーポン債
  - 価格：
  $$P = \sum_{t=1}^N \frac{C}{(1+r)^t} + \frac{X}{(1+r)^N}$$

両方とも割引因子 $(1+r)^t$ を通じて金利 r に依存するため、
**金利上昇 → 債券価格下落**となる。


---

## 金利リスク（Interest Rate Risk）
- 金利が上昇すると、割引率が高くなり債券価格は下落する。
- 金利が低下すると、割引率が低くなり債券価格は上昇する。
- これは、債券価格が将来キャッシュフローの現在価値で決まるためである。




👉 債券価格は**市場金利と逆相関**する。  
👉 これが金利リスクの背景である。


#2-4 デュレーション（Duration）

##デュレーションとは何か

これまで見てきたように、債券価格と市場金利は逆方向に動く。

しかし、
> 金利が1%変化したとき、価格はどれくらい変わるのか？

という問いには、まだ答えられていない。

例えば、

- 満期1年の債券

- 満期20年の債券

この2つは、金利変動に対して同じように動くか？

直感的には、
将来に遠いキャッシュフローほど、割引率の影響を強く受ける。

つまり、
一般に満期が長い債券ほど、金利変動の影響を大きく受ける。

この「金利変動に対する価格の敏感さ」を測る指標が、**デュレーション**である。

---
##マコーレーデュレーション（Macaulay Duration）

デュレーションの最も基本的な定義が、マコーレー・デュレーション（Macaulay Duration） である。

これは、債券の将来キャッシュフローが回収されるまでの「現在価値加重平均時間」を表す。

単なる満期ではなく、各期に受け取るキャッシュフローの現在価値を重みとして時間の平均を取ったものである。

###**数学的定義**

各期のキャッシュフローを $CF_t$とすると、

$$
D_M = \frac{{\Sigma}^{N}_{t=1}t\frac{CF_t}{(1+y)^t}}{P}
$$

- $D_M$：マコーレー・デュレーション

- $P$:債券価格

- $y$:満期利回り（YTM）

- $CF_t$:時点 $t$ に受け取るキャッシュフロー

- $N$:満期までの期数

###**解釈**

マコーレー・デュレーションは、
各期に受け取るキャッシュフローを現在価値に割り引き、
その現在価値を「重み」として時間の平均を取ったものである。

なぜ現在価値を重みとするのか？

将来に遠いキャッシュフローほど、
割引によって現在価値が小さくなるためである。

つまり、

- 早く受け取るお金ほど重みが大きい

- 遅く受け取るお金ほど重みが小さい

という構造になっている。

その結果、デュレーションは

> 「現在の価格を構成しているキャッシュフローが、平均して何年後に回収されるか」

を表す指標になる。
単位は「年」であり、
マコーレー・デュレーションは
**債券の実質的な平均回収期間** と解釈できる。

### **ゼロクーポン特例**

ゼロクーポン債では、満期時にのみキャッシュフローが存在する。
したがって、$D_M = N$


---
##修正デュレーション（Modified Duration）

マコーレー・デュレーションは、「キャッシュフローの平均回収時間」を表す指標であった。

しかし、実務において重要なのは

> 金利が変化したとき、債券価格がどれだけ変動するか

である。

この価格の金利感応度を表す指標が、**修正デュレーション（Modified Duration）** である。


###**数学的定義**

債券価格 $P$ を満期利回り $y$ の関数とするとき、

$$
D_{mod} = -\frac{1}{P}\frac{dP}{dy}
$$

- $D_{mod}$：修正デュレーション
- $P$：債券価格  
- $y$：満期利回り（YTM）




###**解釈**
$\frac{dP}{dy}$ は「金利がわずかに変化したときの価格変化量」。
それを価格で割ることで、$\frac{1}{P}\frac{dP}{dy}$ は「価格変化率」を表す。

マイナス符号は、**金利と価格が逆方向に動く**ことを反映している。


---
##二つのデュレーションの関係

離散複利の場合、

$$
D_{mod} = \frac{D_M}{1+y}
$$


したがって、
- マコーレー：時間の概念

- 修正：価格感応度の概念

という違いがある。

特にゼロクーポン債では $D_M = N$ であるため、

$$
D_{mod} = \frac{N}{1+y}
$$

となる。

---
##価格変化近似式

金利が小さく変化するとき、一次テイラー展開による近似を考えると

$$
\frac{ΔP}{P} ≈ -D_{mod}Δy
$$（Δy が十分小さいと仮定）

つまり、

- デュレーションが大きい

- 金利変化が大きい

ほど、価格変動も大きくなる。


### **投資判断**

また、価格変化近似式より、


- 金利が低下すると予想される場合
→ 価格上昇幅が大きくなるため、
デュレーションの長い債券が有利になりやすい

- 金利が上昇すると予想される場合
→ 価格下落幅を抑えるため、
デュレーションの短い債券が有利になりやすい

デュレーションは金利リスク管理の基本指標である。

#2-5 債券のリスク

## 1. 金利リスク（Interest Rate Risk）

- 債券価格は市場金利の変化により変動する。

- 価格変化の大きさは修正デュレーションで近似できる：


$$
\frac{ΔP}{P} ≈ -D_{mod}Δy
$$
- デュレーションが大きいほど、金利変動に対する価格変動も大きい。

- 満期まで保有すれば契約上のキャッシュフローは受け取れる（デフォルトが発生しない限り）が、途中売却時には価格変動による損益が発生する。

---

## 2. デフォルトリスク（Default Risk）
- 発行体（企業・政府）が利息や元本を支払えないリスク。
- 例：クーポン（年間10000円）が支払われない可能性。
- 信用リスクは利回りスプレッド（国債との差）として価格に反映される。


---

## 3. インフレリスク（Inflation Risk）
- インフレにより将来キャッシュフローの購買力が低下するリスク。
- 名目額を受け取っても実質価値は目減りする。
- 固定利付債は特にインフレリスクに弱い。



#2-6 株式と債券の違い

| 項目 | 株式（Stocks） | 債券（Bonds） |
|---|---|---|
| 性質 | 企業の所有権 | 政府・企業への貸付（負債証券） |
| 投資家の権利 | 議決権あり | 議決権なし |
| 収益の形 | 配当（不確実）＋値上がり益 | 利息（クーポン）または割引差益 |
| 元本返済 | なし | 満期時に元本返済 |
| キャッシュフロー | 不確実 | 比較的確実（デフォルト除く） |
| リスク水準 | 一般に高い | 一般に株式より低い（信用リスクに依存） |
| 価格特性 | 業績・成長期待・金利に依存 | 市場金利・信用リスクに依存 |
| 一般的評価 | ハイリスク・ハイリターン | ローリスク・安定収益（傾向） |
